In [1]:
# Se agregan las librerias del google para realizar la conexion con el drive
from google.colab import files
from google.colab import drive

In [2]:
drive.mount("/content/drive") # Se realiza la peticion para conexion a la cuenta de google drive

Mounted at /content/drive


In [3]:
pathOrigen = r"/content/drive/MyDrive/Diversa/imagenes".replace("\\","/")
path_sr_images_folder = r"/content/drive/MyDrive/Diversa/imagenes_sr".replace("\\","/")

In [5]:
# Clonar el repositorio de Real-ESRGAN y configurar
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr facexlib
!pip install -r requirements.txt
!python setup.py develop

# Descargar los pesos preentrenados
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 759 (delta 0), reused 2 (delta 0), pack-reused 755
Receiving objects: 100% (759/759), 5.39 MiB | 21.05 MiB/s, done.
Resolving deltas: 100% (406/406), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.wh

In [6]:
!python -m pip install torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118 xformers==0.0.21

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 356.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
ERR

In [58]:
import os
import torch
from PIL import Image
import numpy as np
import pandas as pd
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

# Configuración del modelo Real-ESRGAN
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model_path = 'weights/RealESRGAN_x4plus.pth'
upsampler = RealESRGANer(
    scale=4,
    model_path=model_path,
    dni_weight=None,
    model=model,
    tile=0,
    tile_pad=10,
    pre_pad=0,
    half=False,
    gpu_id=None if torch.cuda.is_available() else -1
)

In [60]:
def upscale_image(image_path, output_size=(1500, 1500)):
    """
    Mejora la resolución de la imagen.

    Args:
        image_path (str): Ruta de la imagen de entrada.
        output_size (tuple): Tamaño de la imagen de salida (ancho, alto) en píxeles.

    Returns:
        PIL.Image: Imagen mejorada.
    """
    img = Image.open(image_path).convert('RGB')
    original_width, original_height = img.size

    # Calcular el factor de escala
    scale_x = output_size[0] / original_width
    scale_y = output_size[1] / original_height
    scale = min(scale_x, scale_y)

    # Mejorar la resolución de la imagen
    img = np.array(img)
    output, _ = upsampler.enhance(img, outscale=scale)
    upscaled_img = Image.fromarray(output)

    # Redimensionar la imagen al tamaño especificado
    final_img = upscaled_img.resize(output_size, Image.BICUBIC)
    return final_img

In [61]:
# Se crea una funcion para calcular las metricas de evaluacion entre las imagenes originales y las resultantes
def calculate_metrics(original_image, upscaled_image):
    """
    Calcula PSNR y SSIM entre la imagen original y la mejorada.

    Args:
        original_image (np.array): Imagen original.
        upscaled_image (np.array): Imagen mejorada.

    """
    # Convertir imágenes a arrays numpy
    original = np.array(original_image)
    upscaled = np.array(upscaled_image)

    # Redimensionar la imagen original al tamaño de la imagen mejorada
    original_resized = np.array(original_image.resize(upscaled_image.size, Image.BICUBIC))

    # Calcular PSNR y SSIM
    psnr_value = psnr(original_resized, upscaled)
    ssim_value = ssim(original_resized, upscaled, multichannel=True)

    return psnr_value, ssim_value

In [63]:
input_imagenes_list = os.listdir(pathOrigen) # Listar imágenes

results = []

if not os.path.exists(path_sr_images_folder):
    os.makedirs(path_sr_images_folder)

for image in input_imagenes_list:
    # Ruta de la imagen
    image_path = os.path.join(pathOrigen, image)
    base_name = os.path.basename(image_path)

    # Mejorar la imagen
    hr_image = upscale_image(image_path, output_size=(1500, 1500))
    result_image_name = base_name.split(".")[0] + "_sr"
    new_name = f'{result_image_name}.jpg'
    final = os.path.join(path_sr_images_folder, new_name)
    hr_image.save(final)

    # Calcular métricas
    original_image = Image.open(image_path).convert('RGB')
    psnr_value, ssim_value = calculate_metrics(original_image, hr_image)

    # Guardar resultados en la lista
    results.append({'Imagen': base_name, 'PSNR': psnr_value, 'SSIM': ssim_value})

    print(f'La imagen se ha guardado ---> {final}')
    # print(f'PSNR: {psnr_value:.2f}, SSIM: {ssim_value:.4f}\n')

<ipython-input-61-16d832bef33d>:20: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original_resized, upscaled, multichannel=True)


La imagen se ha guardado ---> /content/drive/MyDrive/Diversa/imagenes_sr/streetview_5_270_sr.jpg
La imagen se ha guardado ---> /content/drive/MyDrive/Diversa/imagenes_sr/streetview_308_90_sr.jpg
La imagen se ha guardado ---> /content/drive/MyDrive/Diversa/imagenes_sr/streetview_274_90_sr.jpg


In [64]:
# Crear un DataFrame a partir de la lista de resultados
pathMetricas = path_sr_images_folder+'/metricas_resultados.csv'
results_df = pd.DataFrame(results)

# Guardar el DataFrame en un archivo CSV
results_df.to_csv(pathMetricas, index=False)